<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Pipeline - AI Avatar

**Tags:** #gsheet #pipeline #automation #snippet #avatar #openai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This notebook only works if you are a Naas maintainer as you need to have the credenditals to send the model to the Workspace API.
<br>
</div>

## Input

### Import libraries

In [ ]:
import naas_data_product
from naas.pipeline import Pipeline, NotebookStep, End
import naas
import os
from IPython.display import Markdown

### Setup variables

In [ ]:
# Avatar meta
avatar_name = "MY AI Avatar"
avatar_image = "https://static.vecteezy.com/system/resources/previews/002/002/257/non_2x/beautiful-woman-avatar-character-icon-free-vector.jpg"
avatar_model_alias = "naas/my-aia-v0.1"
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "ai-characters", avatar_name.lower().replace(" ", "_"), avatar_model_alias.replace("/", "_"))
print("Output directory:", output_dir)

# Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1GDVJxBajYx7VR48yKGnSvMsEjzXvqI94hlmPXI5qEp8/edit#gid=0" or naas.secret.get("MY_AI_AVATAR_SPREADSHEET")
sheet_name = "AIA"

# OpenAI
openai_api_key = naas.secret.get("OPENAI_API_KEY")

## Model

### Setup notebooks
This section is made to declare all the notebooks and create unique IDs that will be used in the pipeline. 

In [ ]:
gsheet = NotebookStep(
    name='Google Sheets', #notebook name display in pipeline
    notebook_path="fine_tuning/Google_Sheet_Fine_tune_dataset_with_OpenAI.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "output_dir": output_dir,
        "spreadsheet_url": spreadsheet_url,
        "sheet_name": sheet_name,
        "openai_api_key": openai_api_key,        
    } #inserted in your notebook after cell tagged 'parameters'
)

supabase = NotebookStep(
    name='Supabase', #notebook name display in pipeline
    notebook_path="fine_tuning/Supabase_Update_available_models.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "output_dir": output_dir,
        "avatar_model_alias": avatar_model_alias,
    } #inserted in your notebook after cell tagged 'parameters'
)

plugin = NotebookStep(
    name='Plugin', #notebook name display in pipeline
    notebook_path="fine_tuning/Naas_Chat_Create_avatar_plugin.ipynb", # notebook path to be run
    parameters={
        "avatar_name": avatar_name,
        "avatar_image": avatar_image,
        "output_dir": output_dir,
    } #inserted in your notebook after cell tagged 'parameters'
)

## Output

### Run pipeline

In [ ]:
pipeline = Pipeline()

pipeline >> gsheet >> supabase >> plugin >> End()

pipeline.run(outputs_path=os.path.join(output_dir, "pipeline_executions"))

### Get plugin

In [ ]:
files = os.listdir(output_dir)
sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(output_dir, x)), reverse=True)
for file in sorted_files:
    if file.startswith("ft:"):
        model = file
        model_id = model.split("::")[1]
        output_path = os.path.join(output_dir, f"{model_id}_plugin.json")
        plugin_url = naas.asset.find(output_path)
        print("Plugin URL:", plugin_url)
        break

### Create new chat
You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [ ]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")